In [1]:
import torch
from src.common.cache import CacheType
import matplotlib.pyplot as plt
import numpy as np
import os
from src.common.cuda_info import get_device
from src.model import PSD_HR
import cv2

TEST_CACHE = CacheType.READ

In [ ]:
from PhythmFormer_Train_in_UBFC_rPPG import \
    T,WIDTH,HEIGHT,BATCH,sbs
MODEL_NAME = "RhythmFormer_Train_in_UBFC_rPPG"
from src.common.cache import CacheType
from src.dataset_reader.UBFC_rPPG import UBFCrPPGsDatasetReader
from src.data_generator.PhythmFormer import PhythmFormerDataGenerator,PhythmFormerDataConfig


TEST_NAME = "Test_in_UBFC_rPPG"
FPS = 30

test_dataset_path = r"/public/share/weiyuanwang/dataset/UBFC-rPPG"
test_cache_path = r"~/cache/RhythmFormer/UBFC-rPPG/test"
test_dataset_reader = UBFCrPPGsDatasetReader(test_dataset_path,dataset=2,dataset_list=[
    'subject1',
    'subject3',
    'subject4','subject5','subject49', 
    'subject8', 'subject9', 'subject48','subject46', 'subject47', 
])
test_dataset_config = PhythmFormerDataConfig(
    cache_root=test_cache_path,
    cache_type=TEST_CACHE,
    generate_num_workers=12,
    step=T,
    width=WIDTH,
    height=HEIGHT,
    slice_interval=T,
    num_workers=12,
    batch_size=BATCH,
    load_to_memory=True
)
test_dataset_generator = PhythmFormerDataGenerator(config=test_dataset_config)
test_raw_data = test_dataset_reader.read() if TEST_CACHE == CacheType.NEW_CACHE else None
test_dataloader = test_dataset_generator.generate(test_raw_data)


dataset size: 109


In [3]:
save_path = f'out/img/test/[{MODEL_NAME}]_[{TEST_NAME}]/item'
os.makedirs(save_path, exist_ok=True)
def draw_pred_true(y_1,y_2,name):
    plt.figure(figsize=(8,6))
    plt.xlabel('Frame')
    plt.ylabel('Strength')
    plt.yticks(np.linspace(0,1,6))
    ax = plt.gca()
    for spine in ax.spines:
        ax.spines[spine].set_visible(False)
    plt.plot(y_1,label='True BVP')
    plt.plot(y_2,label='Predicted BVP')
    plt.legend(loc='upper right')
    plt.savefig(f'{save_path}/{name}.pdf', transparent=True, bbox_inches='tight')
    plt.savefig(f'{save_path}/{name}.svg', transparent=True, bbox_inches='tight')
    plt.close('all')
draw_y = []
draw_y_p = []
min_loss = 10
hr_true_list = list()
hr_pred_list = list()
with torch.no_grad():
    psd_hr_model = PSD_HR(FPS)
    model = sbs.model
    loss_fn = sbs.loss_fn
    device = get_device()
    for i, (X, y) in enumerate(test_dataloader):
        x_tensor = torch.as_tensor(X).float().to(device)
        y_tensor = torch.as_tensor(y).float().to(device)
        for b_index in range(X.shape[0]):
            b_x = x_tensor[b_index].unsqueeze(0)
            b_y = y_tensor[b_index].unsqueeze(0)
            # model for prediction
            hr = psd_hr_model(b_y)
            y_pred = model(b_x)
            hr_pred = psd_hr_model(y_pred)

            loss = loss_fn(y_pred,b_y).item()
            hr_true = hr.squeeze(0).detach().cpu().numpy()
            hr_pred = hr_pred.squeeze(0).detach().cpu().numpy()

            mae = round(np.abs(hr_pred - hr_true),2)
            y_pred = y_pred.squeeze(0).detach().cpu().numpy()
            b_y = b_y.squeeze(0).detach().cpu().numpy()
            # hr list
            hr_true_list.append(hr_true)
            hr_pred_list.append(hr_pred)
            # draw pred
            draw_pred_true(b_y,y_pred,f'{i}_{b_index}_mae={mae:.3}')
            if min_loss > loss:
                min_loss = loss
                draw_y_p = y_pred
                draw_y = b_y

hr_true_list = np.array(hr_true_list)
hr_pred_list = np.array(hr_pred_list)
draw_pred_true(draw_y,draw_y_p,'best')

In [4]:
mae = np.abs((hr_pred_list-hr_true_list)).mean()
mse = ((hr_pred_list-hr_true_list) ** 2 ).mean()
rmse = np.sqrt(mse)
print(f'MAE:{mae}')
print(f'MSE:{mse}')
print(f'RMSE:{rmse}')

MAE:12.955412844036697
MSE:575.4558550458715
RMSE:23.98866096817143
